# Flow Processing Pipeline


## Goal of the pipeline

<font color=red> The goal of this notebook is to associate the raw ADT ground-data for 2013, 2015, 2017 and 2019, and the raw speed ground-data in 2015 to the Aimsun network for the calibration of Aimsun simulations. First the raw data is processed in a common format, then, every detector is associated with a network link inside Aimsun. Later, every detector is associated with a road section to create heatmap and understand the evolution of flows over years. </font>


# TO DO: write this in a better way
### Outputs of the pipeline: 
- One file matching detectors location to Aimsun road section
- One file with the processed flow data for 2013, 2015, 2017, 2019
- One file with the processed speed data for 2015
- One file with the flow data corresponding to road sections for 2013, 2015, 2017 and 2019
- PCA on flow data

### Inputs of the pipeline: 
**Raw data**
- PeMS account [_publicly available_]. Can be created in http://pems.dot.ca.gov
- PeMS detectors location [_publicly available_]. In the dropbox under `Demand/Flow_speed/Traffic\ flow\ studies/PeMS`
- City average annual daily traffic (AADT) data for 2013, 2017 and 2019 [_given by the city of Fremont_]. Located in the dropbox under `Demand/Flow_speed/Traffic\ flow\ studies/City`
- Kimley-Horn flow and speed data for 2015 [_given by the city of Fremont_]. Located in the dropbox under `Demand/Flow_speed/Traffic\ flow\ studies/Kimley\ Horn\ Data`

**Manually made dataset**
- Aimsun network
- Detectors location
- Road section layer
- Doc files or city ADT data corresponding to the PDF files
- Detectors ID to corresponding flow file name


### Temporary files of the pipeline
- CSV flow data
    - City and Kimley Horn
    - PeMS data
- ...

### Dependent scripts: 
- [pems_download.py](https://github.com/Fremont-project/data-processing/blob/master/pipeline/flow/pems_download.py): the script automatically download PeMs data for chosen date.
- [pre_process_flow.py](https://github.com/Fremont-project/data-processing/blob/master/pipeline/flow/pre_process_flow.py): the script (A). parse the ADT data from xlsx, doc and csv files to csv files (B). Find the coordinates of the city detectors (C). Check and adjust the locations of the (City + PeMS) detectors to match them to our network using ArcGIS 
- [process_flow.py](https://github.com/Fremont-project/data-processing/blob/master/pipeline/flow/): this script processes all the flow traffic data into one big CSV file from both city and PeMS data files
- [process_years_together.py](https://github.com/Fremont-project/data-processing/blob/master/pipeline/flow/): this script combines the previous scripts and processes the data for 2013, 2015, 2017, 2019
- fremontdropbox_flow

### Dependent libraries:
- os
- sys
- webbrowser
- time
- requests
- pathlib
- textract
- [numpy](https://numpy.org)
- datetime
- pandas
- To do

### Work done by the script
1. Obtaining Data
    - Turn city pdf to doc
    - PeMS data download
2. Preprocessing city and Kimley-Horn data
    - Parsing the flow data [We are here]
    - Parsing the speed data
    - Geocoding location of the detectors
    - Manually update the location of the detectors
3. Processing the data
    - Creating one file with the flow for all detectors
    - Creating a layer of road section
    - Matching the detectors to road section
    - Creating one file with the flow over time for every road section
    - Matching the detectors with the corresponding road section in Aimsun
4. Exporting the data
    - Exporting a csv file where Aimsun road sections are matched with detectors
    - Exporting a csv file with flow for the different road sections
    - Exporting traffic flow heatmap
    - Exporting speed heatmap
    - PCA on the traffic flow heatmap over years

# <font color=red> TO DO: </font>
- <font color=green> Jiayi, put the to dos in green when done </font>, <font color=blue> in blue if you need some help </font>, <font color=red>put them red if you have not done them yet</font>
- should have exception handler in the process (and python function) if the files are not located where you are looking for.
- run PCA and create heatmap with ArcGIS (heatmap will be done by Theo)
- put the correct github link for the scripts in the Dependent scripts paragraph
- Work done by the script
- Dependent libraries
- Put the scripts together in one main script
- add link from work done by the script to section of the iPython notebook

In [2]:
import sys
import os
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fremontdropbox import get_dropbox_location

dropbox_dir = get_dropbox_location()

## 1. Obtaining Data

### A) PeMS Data Download 
    
Script: pems_download.py <br>
Download traffic data from the PEMS website (pews.dot.ca.gov) for years 2013, 2015, 2017 and 2019 <br>

We download the data calling the method download(detector_ids, year, PeMS_dir) from the pems_download.py file. The process takes about 20 minutes to run. 
Run `print(help(pems.download))` to get some help.

### B) Turn city pdf to doc

Some of the flow data that we got from the city were in pdf files. To be able to parse them, we convert them to doc files using online website. This should be done before running the code. The doc files are inputs of the pipeline.

In [3]:
import pems_download as pems
print(help(pems.download))

Help on function download in module pems_download:

download(year, detector_ids, PeMS_dir)
    This function downloads traffic data from the PeMS website (pems.dot.ca.gov).
    This function has for input:
        - PeMS detectors ID: detector_ids (an array of detectors)
        - Year for the desired data: year (one year as a integer, should be 2013, 2015, 2017 or 2019)
    
    This function has for output:
        - All corresponding PeMS detectors data file for the given year (and the given days encoded in the url).
        - Stored in the download folder as PeMS_dir/PeMS_year/PeMS-ID_YEAR.xlsx (where PeMS-ID is the detector ID given by PeMS).
        One xlsx file has two sheets:
            - PeMS Report Description
            - Report Data
                - Contains the traffic flow data
                - Each row gives the number of vehicles observed in one time step (5 minutes) per lane number over the columns.
                - The first column gives the date and time stamp,

In [4]:
# ************* IMPORTANT *************
# --> For this cell to work, you need to log in to PeMS in the same browser that runs this Jupyter notebook

## The IDs of the PeMS detectors where obtained using ArcGIS software and an input file, pems_detectors.csv, containing the locations of all the PeMS dectectors in California
## this should be done in Python!
detector_ids = [403250, 403256, 403255, 403257, 418387, 418388, 400376,
               413981, 413980, 413982, 402794, 413983, 413984, 413985,
               413987, 413986, 402796, 413988, 402799, 403251, 403710,
               403254, 403719, 400566, 418420, 418419, 418422, 418423,
               402793, 403226, 414015, 414016, 402795, 402797, 414011,
               402798]
PeMS_dir = dropbox_dir + '/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/PeMS'

# pems.download(2013, detector_ids, PeMS_dir)
# pems.download(2015, detector_ids, PeMS_dir)
# pems.download(2017, detector_ids, PeMS_dir)
# pems.download(2019, detector_ids, PeMS_dir)

# TO DO Later:
- Find the list of detectors ID using the project delimitation shapefile. (to be done later)

In [5]:
# ************* TO DO *************
pems.PeMS_tester(2013, PeMS_dir, True)
pems.PeMS_tester(2015, PeMS_dir, True)
pems.PeMS_tester(2017, PeMS_dir, True)
pems.PeMS_tester(2019, PeMS_dir, True)

PeMS during 03/05/2013 00:00:00-03/07/2013 23:59:59
PeMS during 03/03/2015 00:00:00-03/05/2015 23:59:59
PeMS during 03/07/2017 00:00:00-03/09/2017 23:59:59
PeMS during 03/05/2019 00:00:00-03/07/2019 23:59:59


## 2. Parsing city data


### A) Parse city and Kimley Horn flow data from xlsx, doc and csv files to csv files

Here, we process the Excel and PDF ADT data files (city data) into CSV files.

- run `print(help(pre_process.parse_excel_2013)` to get some help for 2013 excel files
- run `print(help(pre_process.parse_excel_2015)` to get some help for 2015 excel files
- run `print(help(pre_process.parse_excel_2017)` to get some help for 2017 excel files
- run `print(help(pre_process.parse_excel_2019)` to get some help for 2019 excel files
- run `print(help(pre_process.parse_adt_as_file)` to get some help for the 2017 and 2019 doc files

In [5]:
import pre_process_flow as pre_process
print(help(pre_process.process_adt_data))

Help on function process_adt_data in module pre_process_flow:

process_adt_data(year, Processed_dir, Input_dir)
    This function processes the Excel and PDF ADT data files (city data) into CSV files. Note that one file corresponds to one main road and the traffic flow data recordings in it.
    
    This function has input:
        - year which takes values 2013, 2015, 2017 or 2019
        - Processed_dir: path to the output
        - Input_dir: path to the inputs
    
    The function has output:
        - CSV files located in the Processed_dir/Year_processed/ folder where Year=2017 or 2019
    
    For the function to work:
        - Files should be located in 
            1. Input_dir/Year\ EXT/ folder if Year=2013, 2017 or 2019 where:
                a. Year=2013, 2017 or 2019 if Ext=ADT Data 
                b. Year=2017 or 2019 if Ext=doc for 2017 and 2019
            2. Input_dir/Raw\ data/ folder if Year=2015

None


# <font color=red> TO DO: </font>
- <font color=green> Re do the parsing such that all the files look the same. The time format should be the same everywhere.
- Remove the unnecessary functions in pre_process
- <font color=green> Write test to check if the parsing of doc file is correct

In [6]:
ADT_dir = dropbox_dir + '/Private Structured data collection/Data processing/Raw/Demand/Flow_speed'
Processed_dir = dropbox_dir + '/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/Flow_processed'
City_dir = ADT_dir + "/City"
Kimley_Horn_flow_dir = ADT_dir + "/Kimley Horn Data"

# pre_process.process_adt_data(2013, Processed_dir, City_dir)
# pre_process.process_adt_data(2015, Processed_dir, Kimley_Horn_flow_dir)
# pre_process.process_adt_data(2017, Processed_dir, City_dir)
# pre_process.process_adt_data(2019, Processed_dir, City_dir)



### B) Find the coordinates of the city detectors
We obtained the coordinates of the detectors by calling the method get_geo_data(year) from the pre_process.py python file. 

- run `print(help(pre_process.get_geo_data))` to get some help.

# <font color=red> TO DO: </font>
- Put the different files year_info.csv together, and compare it to the [google doc](https://docs.google.com/spreadsheets/d/1tcps-8aorPZLY8nswnNCmjWSJi-7ey8Ps4twWFz2ls0/edit#gid=0)
- Make sure to process doc files
- Make sure to process PeMS files too (for this ask Theo)
- Then we want to match the detectors to the ID associated to them here.

In [ ]:
#pre_process.get_geo_data(2013, City_dir, Processed_dir)
#pre_process.get_geo_data(2015, Kimley_Horn_flow_dir, Processed_dir)
# pre_process.get_geo_data(2017, City_dir, Processed_dir)
# pre_process.get_geo_data(2019, City_dir, Processed_dir)

In [10]:
import pandas as pd
all_filenames = []
for filename in os.listdir(Processed_dir):
    _, file_ext = os.path.splitext(filename)
    if file_ext == '.csv':
        print(filename)
        all_filenames.append(filename)
combined_csv = pd.concat([pd.read_csv(Processed_dir + "/" + str(f)) for f in all_filenames])
combined_csv.to_csv(Processed_dir + "/" + "combined_csv.csv", index=False, encoding='utf-8-sig')

2013_info_coor.csv
2017_info_coor.csv
2019_info_coor.csv
2015_info_coor.csv


In [22]:
import csv
debug = False
fout=open(Processed_dir + "/" + "out.csv","a")
id_counter = 1
for filename in os.listdir(Processed_dir):
    file_name, file_ext = os.path.splitext(filename)
    if file_ext == '.csv' and file_name != 'out':
        if debug == True:
            print(filename)
        curr_year = filename.split("_")[0]
        curr_opening = "./" + curr_year + " ADT Data"
        f = open(Processed_dir + "/" + filename)
        fout.write('\n')
        reader = csv.reader(f)
        header= next(reader)
        header = header[1:]
        opening_line = ','.join(["Id", curr_opening] + header)
        if debug == True:
            print(opening_line)
        fout.write(opening_line)
        for line in f:
            first_ele = line.split(",")[0]
            if first_ele == 'Name': #This is header
#                 header = line[len(first_ele)+1:]
#                 header = "Id," + curr_opening + "," + header
#                 print(header)
#                 fout.write(header)
                continue
            line = str(id_counter) + "," + line
            id_counter += 1
            if debug == True:
                print(line)
            fout.write(line)
        print("done with " + str(filename))
fout.close()


Id,./2013 ADT Data,City,Main road,Cross road,Start lat,Start lng,End lat,End lng
done with 2013_info_coor.csv
Id,./2017 ADT Data,City,Main road,Cross road,Start lat,Start lng,End lat,End lng
done with 2017_info_coor.csv
Id,./2019 ADT Data,City,Main road,Cross road,Start lat,Start lng,End lat,End lng
done with 2019_info_coor.csv
Id,./2015 ADT Data,City,Main road,Cross road,Start lat,Start lng,End lat,End lng
done with 2015_info_coor.csv


In [57]:
fout=open(Processed_dir + "/" + '2017_info_coor.csv')

In [60]:
for line in fout:
    print(line)

Name,City,Main road,Cross road,Start lat,Start lng,End lat,End lng

MISSION BLVD BT DURHAM RD AND CURTNER RD.xlsx,Fremont,Mission Blvd,Durham Rd And Curtner Rd,37.5136269,-121.9194106,37.4972833,-121.9206722

PASEO PADRE PKWY BT CAMINO DEL CAMPO AND PINE ST.xlsx,Fremont,Paseo Padre Pkwy,Camino Del Campo And Pine St,37.5205218,-121.9251071,37.5235673,-121.9284821

GRIMMER BLVD BT OSGOOD RD AND FREMONT BLVD.xlsx,Fremont,Grimmer Blvd,Osgood Rd And Fremont Blvd,37.50526139999999,-121.9377826,37.5364522,-121.9655703

DRISCOLL RD BT COMAC TERRACE AND DENISE ST.xlsx,Fremont,Driscoll Rd,Comac Terrace And Denise St,37.5417999,-121.9475293,37.5379431,-121.9512488

PAGE AVE BT KATO RD AND MILMONT DR.xlsx,Fremont,Page Ave,Kato Rd And Milmont Dr,37.467891,-121.9278693,37.4688514,-121.9251185

WARREN AVE BT CURTNER RD AND WARM SPRINGS BLVD.xlsx,Fremont,Warren Ave,Curtner Rd And Warm Springs Blvd,37.4956742,-121.9141113,37.4871224,-121.9280083

PASEO PADRE PKWY BT MISSION BLVD AND CURTNER RD.xlsx,Fre

# Theo: I am here

### C) Check and adjust the locations of the (City + PeMS) detectors to match them to our network using ArcGIS <br>
Done in the software manually in ArcGIS.
1. Export Aimsun network as GIS file
2. Import Aimsun network in ArcGIS
3. Import detectors in ArcGIS as XY_points
4. Move detectors to put them on corresponding road in Aimsun
5. Associate to every detectors the External ID of the Aimsun road (to be done again)

**TO DO THEO**: Add the process to create the detectors inside Aimsun.
Add the process to match the detectors to road section (and create the file lines_to_detectors.xlsx


### Later to do: do the spatial join in python

### 3. Process Data

In [ ]:
import process_flow as pf

# <font color=red> TO DO: </font>
- Create the file Flow_processed_tmp.csv from year_info.csv to check if everything is fine
- Rename the file Flow_processed_tmp.csv -> detectors_id.csv and put it in the manually made dataset
- Merge flow_processed_city.csv and flow_processed_PeMS.csv together in the function
- Go over Zixuan code to make the code in process_flow better

### A) Process the csv files (city + caltrans) to one big file. <br>
source code: processing flow to one CSV.ipynb <br>

We combine all the flow traffic data into one big CSV file from both city and PeMS data files. This is done by calling the function process_data() from the process_flow.py python file. 

Description of function process_data()
<br>
The function has input:
- "Flow_processed_tmp.csv" file that lists all the processed files from city and PeMS data
- The processed files created from the Parsing Data section

The function has output:
- "Flow_processed_city.csv" containing combined city flow data for all year
- "Flow_processed_PeMS.csv" containing combined PeMS flow data for all years

For the function to work:
- The processed files (input) must be located in City and PeMs folders
- 2013 city processed files are located in "City/2013 reformat/"
- 2017 and 2019 city processed files that originated from DOC (which originated from PDF) files are located in "City/Year reformat/Format from pdf" folder where Year=2017 or 2019
- 2017 and 2019 city processed files that originated from Excel files are located in "City/Year reformat/Format from xlsx" folder where Year=2017 or 2019
- 2013, 2017 and 2019 PeMS data files are located in "PeMS_Year" folder where Year=2013, 2017 or 2019

For the pipeline to work:
- The ouput files must remain in the working directory, no moving necessary.

Structure of ouput files: 
- Flow_processed_city.csv
    - contains city traffic data where the rows represent traffic flow. The first 5 columns give info about the traffic flow and are Year, Name, Id, Direction, Day 1 where Name refers to the file name from which the data originated, Id is the Id from the "Flow_processed_tmp.csv" file, Direction is the direction of flow and Day 1 is the start date of recording. The columns that follow are day-timesteps for flow data. There are 3 days total over which traffic flow is recorded and time progresses in 15 minute steps. Hence the data columns progress as "Day 1 - 0:0", "Day 1 - 0:15", "Day 1 - 0:30",...,"Day 3 - 23:30", "Day 3 - 23:45".
- Flow_processed_PeMS.csv
    - contains PeMS flow traffic data where the rows represent traffic flow. The first columns are Name, Id and Name PeMS where Name contains the PeMS detector Id, Id is the Id assigned from "Flow_processed_tmp.csv", Name PeMs is the road address. The next 6 columns give Observed Year and Day Year for the 3 years, 2013, 2017 and 2019. Observed Year is the percentage of the observed data and Day Year is the start date of recording. The columns that follow are Year-Day-timestep, there are 3 years, 3 days and time progresses in 15 minute steps. Hence the columns progress as "2013-Day 1 - 0:0", "2013-Day 1 - 0:15", "2013-Day 1 - 0:30",...,"2019-Day 3 - 23:30", "2019-Day 3 - 23:45".

**(DONE) TO DO 8**: Explain the structure of the output files. Also, feel free to document the doc in the python file (or iPython file). Explain also the input (Flow_processed_tmp.csv) and how it was created (I think it was created from the google doc https://docs.google.com/spreadsheets/d/1tcps-8aorPZLY8nswnNCmjWSJi-7ey8Ps4twWFz2ls0/edit#gid=0).

***Edson Question***: the Flow_processed_tmp.csv file and the google doc seem the same to me (except for the lat, lng info on the right side of the google doc). Beyond this, I don't know how the file was created. Who to ask for more info?

***Theo Answer***: I guess I have created the file from the google doc. But I have also created the google doc during the parsing. Probably in 2) we can create Flow_processed_tmp.csv from year_info.csv

# added 2015 year

In [ ]:
import process_flow as pf
pf.process_data()

### B) Create file that gives traffic flows for specific road sections for every year. <br>
source note: put years together.ipynb

- Use detectors (lines_to_detectors.csv) and flow processed city (flow_processed_city.csv) data to create all years flow data in "flow_processed_section.csv"
- Note that the erroneous files are still being skipped, they are: ['DurhamRd I680 MissionBlv EB', 'Mission blvd Pine Washington SB']

# To do 9: the function pytogether should be written again. + add 2015
- add the PeMS data
- be more clever about missing data or road section associated with several detectors (take the average)

### To do later, do the spatial join in python

In [ ]:
import process_years_together as pytogether

line_to_detectors = 'lines_to_detectors.csv'
flow_processed_city = 'Flow_processed_city.csv'
pytogether.run(line_to_detectors, flow_processed_city)

### 4. Data Analysis

# TO DO: To be done after the other to dos has been done.

A) PCA

B) Analyse PCA results using heatmap inside ArcGIS